In [8]:
!pip install transformers
!pip install scikit-learn
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=5ffc6e372ae167fb2e252d8d7633a9ad331890ce185d47baca8d53cd6014646b
  Stored in directory: /root/.cache/pip/wheels/d5/b2/a9/590d15767d34955f20a9a033e8db973b79cb5672d95790c0a9
Successfully built sklearn


In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install emoji --quiet

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import re
import emoji

In [13]:
import os
import glob
path = r"/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset-master/Natural-Hazards-Twitter-Dataset-master/*.csv" # use your path
all_files = glob.glob(path)
all_files

filenames=[]
for p in all_files:
    filenames.append(p.split("/")[-1])

df_dict = dict()
for n,f in zip(filenames,all_files):
    df_dict[n] = pd.read_csv(f)
    if n=='2018Michael_Summary.csv':
      df_dict[n]['num']=df_dict[n]['Unnamed: 0']
      df_dict[n]['label']=df_dict[n]['sentiment']
      df_dict[n].drop(['sentiment','Unnamed: 0'],axis=1,inplace=True)
    df_dict[n]['filename']=[n]*len(df_dict[n])




In [14]:

df_dict.keys()


dict_keys(['2012Sandy_Summary.csv', '2011Tornado_Summary.csv', '2016Blizzard_Summary.csv', '2017Hurricane_Summary.csv', '2016Matthew_Summary.csv', '2013Floods_Summary.csv', '2018Michael_Summary.csv', '2018Wildfires_Summary.csv', '2019Dorian_Summary.csv'])

In [15]:


#df_all=pd.concat(df_dict.values(), ignore_index=True) 

# 2023/0425/0322
# df_all=df_dict['2011Tornado_Summary.csv']
# required_df=df_all[['text','label','filename']]
# required_df.head()

def clean_text(line):
  clean_text=re.sub(r'(https?://)?([\da-z.-]+).([a-z.]{2,6})([/\w .-])','',line)
  clean_text = re.sub("[^a-z0-9]"," ", clean_text)
  clean_text = re.sub("@[A-Za-z0-9]+","",clean_text) #Remove @ sign
  clean_text = " ".join(clean_text.split())
  clean_text = ''.join(c for c in clean_text if c not in emoji.EMOJI_DATA) #Remove Emojis
  clean_text = clean_text.lower()
  return clean_text

# 2023/0425/0322  
# required_df['text']=required_df['text'].apply(lambda x: clean_text(x))



# # Replace this with the path to your CSV file
# csv_path = "/content/drive/MyDrive/predata.csv"


# data = pd.read_csv(csv_path)

# Add the function to convert sentiment polarity to labels

# def label_sentiment(polarity):
#     if polarity > 0:
#         return "positive"
#     elif polarity < 0:
#         return "negative"
#     else:
#         return "neutral"

# Apply the function to create a new column with sentiment labels
# data["sentiment_label"] = data["sentiment_polarity"].apply(label_sentiment)

# 2023/0425/0322
# texts = required_df["text"].tolist()
# sentiments = required_df["label"].tolist()

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")



In [16]:
def encode(texts, labels):
    input_ids = []
    attention_masks = []

    # Create a dictionary to map labels to integers
    label_map = {"positive": 0, "negative": 1, "neutral": 2}
    
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=512, truncation=True, padding="max_length", return_tensors="pt"
        )
        input_ids.append(encoded_dict["input_ids"])
        attention_masks.append(encoded_dict["attention_mask"])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    # Convert the labels to integers using the label_map dictionary and then to floats
    labels = torch.tensor([label for label in labels], dtype=torch.float)

    return input_ids, attention_masks, labels




In [17]:
# 2023/0425/0322
# train,test = train_test_split(required_df, test_size=0.7, shuffle=False)


In [18]:

# 2023/0425/0322
# train_input_ids, train_attention_masks, train_labels = encode(train['text'], train['label'])
# test_input_ids, test_attention_masks, test_labels = encode(test['text'], test['label'])


In [19]:
# 2023/0425/0322
# train_labels

In [20]:
class MyClassificationDataset(torch.utils.data.Dataset):
    #'input_ids', 'attention_mask', 'label'
    def __init__(self, data):
        text, labels = data['text'].values,data['label'].values
        text = [str(i) for i in data['text'].values]
        self.examples = tokenizer(text,truncation=True,padding="max_length",
                                  return_tensors="pt")
        self.labels = torch.tensor(labels, dtype=torch.float)


    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, index):
        return {'input_ids': self.examples['input_ids'][index],
                'attention_mask':self.examples['attention_mask'][index],
                'labels':self.labels[index],}


In [21]:

# 2023/0425/0322
# train_dataset = MyClassificationDataset(train)
# test_dataset = MyClassificationDataset(test)


In [22]:

# 2023/0425/0322 added
def process_file(df):
  df['text'] = df['text'].apply(lambda x: clean_text(x))
  texts = df["text"].tolist()
  sentiments = df["label"].tolist()

  train, test = train_test_split(df, test_size=0.7, shuffle=False)

  train_input_ids, train_attention_masks, train_labels = encode(train['text'], train['label'])
  test_input_ids, test_attention_masks, test_labels = encode(test['text'], test['label'])

  train_dataset = MyClassificationDataset(train)
  test_dataset = MyClassificationDataset(test)


  training_args = TrainingArguments(
      output_dir="./results",
      num_train_epochs=10,
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      warmup_steps=500,
      weight_decay=0.01,
      logging_dir="./logs",
      #logging_steps=10,
      evaluation_strategy='steps',
      load_best_model_at_end=True,
      report_to="none"  # Add this line to ensure metrics are logged
  )
  model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=1)

  import numpy as np

  from sklearn.metrics import accuracy_score, precision_recall_fscore_support

  def compute_metrics(pred):
      labels = pred.label_ids
      preds = np.round(pred.predictions.argmax(-1))

      precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
      acc = accuracy_score(labels, preds)
      return {"eval_accuracy": acc, "eval_precision": precision, "eval_recall": recall, "eval_f1": f1}

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  pred,labels,metrics=trainer.predict(test_dataset,metric_key_prefix="predict")

  from sklearn.metrics import f1_score
  precision, recall, f1, _ = precision_recall_fscore_support(labels,np.round(pred),average="binary",zero_division=0)
  acc = accuracy_score(labels, np.round(pred))

  # 2023/0425/0322 added
  return precision, recall, f1, acc

In [23]:
  # labels

In [24]:
  # pred

In [25]:
# 2023/0425/0322
# np.round(pred)
# print(precision)
# print(recall)
# print(f1)
# print(acc)

In [26]:
# 2023/0425/0322 added
for filename, df in df_dict.items():
  df = df[['text', 'label', 'filename']]
  precision, recall, f1, acc = process_file(df)
  print(f"Results for {filename}:")
  print(f"Precision: {precision}")
  print(f"Recall: {recall}")
  print(f"F1: {f1}")
  print(f"Accuracy: {acc}")
  print("\n")


# eval_results = trainer.evaluate()
# accuracy = eval_results["eval_accuracy"]
# precision = eval_results["eval_precision"]
# recall = eval_results["eval_recall"]
# f1_score = eval_results["eval_f1"]

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1_score:.4f}")


<ipython-input-22-778c51f09e7a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: clean_text(x))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.175200,0.300077,0.317678,0.000000,0.000000,0.000000


Results for 2012Sandy_Summary.csv:
Precision: 0.6804049168474331
Recall: 0.8996175908221797
F1: 0.7748044462741869
Accuracy: 0.6431833007175473




<ipython-input-22-778c51f09e7a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: clean_text(x))
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification 

...

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.064100,0.080994,0.088564,0.000000,0.000000,0.000000
1000,0.035300,0.085243,0.088564,0.000000,0.000000,0.000000
1500,0.015700,0.087220,0.088564,0.000000,0.000000,0.000000


Results for 2018Wildfires_Summary.csv:
Precision: 0.9114356743318831
Recall: 1.0
F1: 0.9536660705576329
Accuracy: 0.9114356743318831




<ipython-input-22-778c51f09e7a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: clean_text(x))
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification 

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.191500,0.172885,0.189876,0.000000,0.000000,0.000000
1000,0.131800,0.167664,0.189876,0.000000,0.000000,0.000000
1500,0.080000,0.200202,0.189876,0.000000,0.000000,0.000000
2000,0.052400,0.212701,0.189876,0.000000,0.000000,0.000000
2500,0.041600,0.211518,0.189876,0.000000,0.000000,0.000000


Results for 2019Dorian_Summary.csv:
Precision: 0.8222222222222222
Recall: 0.9594961718942949
F1: 0.8855710052427627
Accuracy: 0.7991196478591437


